In [ ]:
#Todo Taking responce from onwer and tell to visitor
#Face recog accuracy increase
#Visitor comming time
#Save visitor data with time

from email.message import EmailMessage
import face_recognition
import cv2
import pyttsx3
import datetime
import gtts
import time
from playsound import playsound
import smtplib
import imghdr
import os
import numpy as np
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import email.mime
import keyboard
from pynput.keyboard import Key, Listener 
import keyboard
import speech_recognition as sr 
import pyaudio
from email.message import EmailMessage
    
def speech_to_text():
    r = sr.Recognizer() 
    
    def SpeakText(command):
        engine = pyttsx3.init()
        engine.say(command) 
        engine.runAndWait()
    
    while(1):    
        try:
            with sr.Microphone() as source2:

                r.adjust_for_ambient_noise(source2, duration=0.2)

                audio2 = r.listen(source2)

                MyText = r.recognize_google(audio2)
                MyText = MyText.lower()

#                 print("Did you say "+MyText)
                return MyText

        except sr.RequestError as e:
#             print("Could not request results; {0}".format(e))
            MyText='No response'
            return MyText

        except sr.UnknownValueError:
#             print("unknown error occured")
            MyText='No response'
            return MyText
    
def greeting():
    curr_time = datetime.datetime.now()
    curr_time = str(curr_time.hour).zfill(2)+str(curr_time.minute).zfill(2)
    if(curr_time>'0000' and curr_time<='1159'):
        engine = pyttsx3.init() 
        engine.say("good morning") 
        engine.runAndWait()
    elif(curr_time<='1700'):
        engine = pyttsx3.init() 
        engine.say("good afternoon")
        engine.runAndWait()
    else:
        engine = pyttsx3.init() 
        engine.say("good evening")
        engine.runAndWait()
        
def capture_image():
    videoCaptureObject = cv2.VideoCapture(0)
    result = True
    while(result):
        ret,frame = videoCaptureObject.read()
        cv2.imwrite("./visitor/visitor.jpg",frame)
        result = False
    videoCaptureObject.release()
    cv2.destroyAllWindows()
    
def email(msg, ph):
    Sender_Email = "apnidunia276@gmail.com"
    Reciever_Email = "apnidunia276@gmail.com"
    Password = 'rrrkuuhetwhrsili'

    newMessage = EmailMessage()                         
    newMessage['Subject'] = "Unknown person came" 
    newMessage['From'] = Sender_Email                   
    newMessage['To'] = Reciever_Email                   
    newMessage.set_content('Do you want to let him in?\nReason: ' + msg + '\nPhone number : ' + ph)

    with open('./visitor/visitor.jpg', 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

        smtp.login(Sender_Email, Password)              
        smtp.send_message(newMessage)
        
def findEncodings(images):
    encodeList = []
    for i in images:
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(i)[0]
        encodeList.append(encode)
    return encodeList

path = 'TestImages'
images = [] 
classNames = []
myList = os.listdir(path)
print(myList) 
for i in myList:
    curImg = cv2.imread(f'{path}/{i}')
    images.append(curImg)
    classNames.append(os.path.splitext(i)[0])
print(classNames)
encodeListKnown = findEncodings(images)
print('Encoding Complete')

def recog():
    capture_image()
    
    cur = cv2.imread('./visitor/visitor.jpg')
    imgSmall = cv2.resize(cur,(0,0),None,0.25,0.25)
    imgSmall = cv2.cvtColor(imgSmall, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgSmall)
    encodesCurFrame = face_recognition.face_encodings(imgSmall,facesCurFrame)

    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):

        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)

        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            engine = pyttsx3.init() 
            engine.say("welcome") 
            engine.say("door unlocked") 
            engine.runAndWait()
            return name
        
        else:
            name='Unknown'
            engine = pyttsx3.init() 
            engine.say("unknown visitor asking for permission") 
            engine.say("please tell the reason for visiting") 
            engine.runAndWait()
            msg = speech_to_text()
            print(msg)
            engine = pyttsx3.init() 
            engine.say("please enter phone number through keyboard") 
            engine.runAndWait()
            ph = input("Enter phone number : ")
            email(msg, ph)
            return name

while True:
    keyboard.wait("e")
    greeting()
    name = recog()
    print(name)


['Sam test.jpg', 'Subham test.jpg']
['Sam test', 'Subham test']
Encoding Complete
None
None
None
None
to meet mr aman
Enter phone number : 123
Unknown


In [1]:
import smtplib
import imghdr
from email.message import EmailMessage

Sender_Email = "apnidunia276@gmail.com"
Reciever_Email = "apnidunia276@gmail.com"
Password = 'rrrkuuhetwhrsili'

newMessage = EmailMessage()                         
newMessage['Subject'] = "Check out the new logo" 
newMessage['From'] = Sender_Email                   
newMessage['To'] = Reciever_Email                   
newMessage.set_content('Let me know what you think. Image attached!') 

with open('./visitor/visitor.jpg', 'rb') as f:
    image_data = f.read()
    image_type = imghdr.what(f.name)
    image_name = f.name

newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    
    smtp.login(Sender_Email, Password)              
    smtp.send_message(newMessage)

In [4]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import gtts
import time
from playsound import playsound
import smtplib
import imghdr
import pyttsx3
import smtplib

# matches = 0
# # faceDis = 0
# def email():
#     sender = 'abhibansal@gmail.com'
#     receivers = ['lxshubhamsingh@gmail.com']
#     message = 'Hello'
#     try:
#         smtpObj = smtplib.SMTP('localhost')
#         smtpObj.sendmail(sender, receivers, message)         
#         print ("Successfully sent email")
#     except SMTPException:
#         engine.say("Error: unable to send email")

def findEncodings(images):
    encodeList = []
    for i in images:
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(i)[0]
        encodeList.append(encode)
    return encodeList

# def search(name):
#     images = []
#     classNames = []
#     path = 'TestImages'
# myList = os.listdir(path)
#     for i in myList:
#         curImg = cv2.imread(f'{path}/{i}')
#         images.append(curImg)
#         classNames.append(os.path.splitext(i)[0])
        
#     for i in range(len(classNames)):
#         if classNames[i] == name:
#             return True
#     return False

def face_recog():
    path = 'TestImages'
    images = [] #List of images
    classNames = [] #For names of all images
    myList = os.listdir(path)

    print(myList) #Store list of all images

    for i in myList:
        curImg = cv2.imread(f'{path}/{i}')
        images.append(curImg)
        classNames.append(os.path.splitext(i)[0])
    print(classNames)

    encodeListKnown = findEncodings(images)
    print('Encoding Complete')

    cap = cv2.VideoCapture(0)

    while True:
        success, img = cap.read()
        #img = captureScreen()
        imgSmall = cv2.resize(img,(0,0),None,0.25,0.25)
        imgSmall = cv2.cvtColor(imgSmall, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgSmall)
        encodesCurFrame = face_recognition.face_encodings(imgSmall,facesCurFrame)
#         global matches
#         global faceDis
        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
#             global faceDis
            matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
            # print(faceDis)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
#                 print(name)
                return name

            else:
                name='Unknown'
                return name
#                 print(name)

#                 y1,x2,y2,x1 = faceLoc
#                 y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
#                 cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
#                 cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
#                 cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
#         cv2.imshow('Webcam',img)
#         cv2.waitKey(1)
#     return name
                   
def greeting():
    curr_time = datetime.datetime.now()
    curr_time = str(curr_time.hour).zfill(2)+str(curr_time.minute).zfill(2)
    if(curr_time>'0000' and curr_time<='1159'):
        engine = pyttsx3.init() 
        engine.say("good morning") 
    elif(curr_time<='1700'):
        engine = pyttsx3.init() 
        engine.say("good afternoon")
    else:
        engine = pyttsx3.init() 
        engine.say("good evening")
            
def image_save():
    cam = cv2.VideoCapture(0)
    result, image = cam.read()
    engine = pyttsx3.init()
    if result:
        cv2.imwrite("unknown_commer.png", image)
        engine = pyttsx3.init() 
        engine.say("asking for permission please wait")
#         email()
        waitKey(0)
    else: 
        engine.say("please write your contact number onwer will contact later")
        
engine = pyttsx3.init() 
engine.say("Hello sir") 
engine.say("Identifying") 
engine.runAndWait()

name = face_recog()
print(name)

if(name!='unknown'):
    engine.say("Welcome sir")
    engine.runAndWait()
else:
    image_save()

['Aman test.jpg', 'Sam test.jpg', 'Subham test.jpg']
['Aman test', 'Sam test', 'Subham test']
Encoding Complete
AMAN TEST



# def detect():
#     while True:
#         try:
#             if keyboard.is_pressed('q'):
#                 print('key pressed')
#                 return True
# #             else:
# #                 break
#         except:
#             break

In [ ]:
import speech_recognition as sr
import pyttsx3 
import pyaudio
  
r = sr.Recognizer() 
  
def SpeakText(command):
      
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command) 
    engine.runAndWait()
    
while(1):    
    try:
        with sr.Microphone() as source2:

            r.adjust_for_ambient_noise(source2, duration=0.2)

            audio2 = r.listen(source2)

            MyText = r.recognize_google(audio2)
            MyText = MyText.lower()
  
            print("Did you say "+MyText)
            SpeakText(MyText)
              
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
          
    except sr.UnknownValueError:
        print("unknown error occured")

Did you say hello hello hello hello
Did you say did you say hello hello hello hello
unknown error occured


In [ ]:
def email(msg, ph):
    img = cv2.imread('./visitor/visitor.jpg')
    
    Subject = "Unknowm visitor came !"
    Body = "Do you want to let him in ?"
    Message = f'Subject: {Subject}\n\n{Body}\n\n Reason: {msg}\n\n Phone number: {ph}'
    
    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        smtp.login("apnidunia276@gmail.com", "rrrkuuhetwhrsili")
        smtp.sendmail("apnidunia276@gmail.com", "apnidunia276@gmail.com", Message)
    
#     s = smtplib.SMTP('smtp.gmail.com', 587)
#     s.set_debuglevel(0)
#     s.ehlo()
#     s.starttls()
#     s.login("apnidunia276@gmail.com", "rrrkuuhetwhrsili")
#     rec = ['amanchaturvedi87@gmail.com', 'abhibansal276@gmail.com']
#     message = "Reason : " + msg + " Phone number : " + ph
#     print(message)
#     s.sendmail("apnidunia276@gmail.com", rec, message)
# #     print("success")
#     s.quit()




def email(msg, ph):
    img = cv2.imread('./visitor/visitor.jpg')
    
    Subject = "Unknowm visitor came !"
    Body = "Do you want to let him in ?"
    Message = f'Subject: {Subject}\n\n{Body}\n\n Reason: {msg}\n\n Phone number: {ph}'
    
    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        smtp.login("apnidunia276@gmail.com", "rrrkuuhetwhrsili")
        smtp.sendmail("apnidunia276@gmail.com", "apnidunia276@gmail.com", Message)

In [2]:
#Worksheet one

import math #Importing math module for math calculations

print("Enter the coefficients of the form ax^3 + bx^2 + cx + d")

lst=[] #Creating list for storing coefficients

for i in range(0,4): #Using for loop we are asking for coefficients and storing them in list
    a = int(input(f"Enter {i}th coefficient : ")) #Asking for coefficient from user and storing it in a
    lst.append(a) #Appending (storing) value of a in the list

x=int(input("Enter the value of x:")) #Asking value of a

sum1=0 #Initializing a variable sum1 with 0

j=3

for i in range(0,3): #Using this for loop we are calculating expression value
    while(j>0): #Using while loop we are evaluating the value of polynomial
        sum1=sum1+(lst[i]*math.pow(x,j)) #Calculating expression value and updating sum variable value
        break
    j=j-1

sum1=sum1+lst[3] 

print(f"The value of the polynomial is : {sum1}") #Printing calculated value

Enter the coefficients of the form ax^3 + bx^2 + cx + d
Enter 0th coefficient : 4
Enter 1th coefficient : 6
Enter 2th coefficient : 4
Enter 3th coefficient : 8
Enter the value of x:2
The value of the polynomial is : 72.0
